# Create local Neo4j Knowledge Graph (for testing)
This notebook is provided to test creating a knowledge graph using a locally installed version of Neo4j.

This notebook creates a local version of the knowledge graph by ingesting the .csv files in the Neo4j import directory.

Note: This notebook works only on Linux and Mac OS.

The path `NEO4J_HOME` must be set to the Neo4j installation directory path.

In [33]:
import os
import time
from pathlib import Path

In [34]:
NEO4J_HOME = Path(os.getenv('NEO4J_HOME'))
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


### Start Neo4j database if it is not running

In [35]:
status = !"$NEO4J_HOME"/bin/neo4j status
status = str(status)
if not 'Neo4j is running' in status:
   !"$NEO4J_HOME"/bin/neo4j start

Directories in use:
  home:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3
  config:       /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/conf
  logs:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/logs
  plugins:      /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/plugins
  import:       /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3/import
  data:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a2

#### Wait until Neo4j has started up

In [36]:
status = !"$NEO4J_HOME"/bin/neo4j status
status = str(status)
while not 'Neo4j is running' in status:
    time.sleep(15)
    status = !"$NEO4J_HOME"/bin/neo4j status
    status = str(status)
    print(status)

# wait until neo4j is ready to receive requests
time.sleep(60)
print(status)

['Neo4j is running at pid 11137']


##### NOTE: Starting Neo4j from this notebook doesn't always work. If the line printed above is `Neo4j is not running`, go back to the top of the Notebook and try starting it again.

### Run Cypher commands to create Knowledge Graph

In [37]:
start = time.time()

In [38]:
# TODO create a batch script for Windows
#!./run_cyphers.sh
!../../scripts/run_cyphers.sh

Endpoint: bolt://localhost:7687
Username: neo4j
Password: neo4jbinder
 
----------------------------------------------
Running 00a-Init.cypher:
 
// delete all nodes
MATCH (n) DETACH DELETE n;

// delete all constraints and indices
CALL db.index.fulltext.drop('locations');
CALL db.index.fulltext.drop('bioentities');
CALL apoc.schema.assert({},{});


// create full text search indices
CALL db.index.fulltext.createNodeIndex('locations',['World', 'UNRegion', 'UNSubRegion', 'UNIntermediateRegion', 'Country', 'Admin1', 'Admin2', 'USRegion', 'USDivision', 'City', 'CruiseShip', 'PostalCode','Tract'],['name', 'iso', 'iso3', 'fips', 'geoId']);
CALL db.index.fulltext.createNodeIndex('bioentities',['ProteinName', 'Protein', 'Gene', 'Strain', 'Variant', 'Organism', 'Outbreak'],['name', 'scientificName', 'taxonomyId', 'proteinVariant', 'variantType', 'variantConsequence']);
                                                                                            
// create constraints and indices

In [39]:
end = time.time()
print('time:', end-start)

time: 403.0918629169464


### Make sure to shutdown Neo4j!

In [40]:
!"$NEO4J_HOME"/bin/neo4j stop

Stopping Neo4j....... stopped
